# 要約 
このJupyter Notebookは、「LMSYS - Chatbot Arena人間による好み予測チャレンジ」に参加するための機械学習モデルの構築プロセスを示しています。主な目的は、2つの異なる言語モデル（LLM）が生成した応答の中から、どちらがユーザーに好まれるかを予測することです。

### 問題の取り組み
本Notebookでは、提供されたトレーニングデータを使用して、ユーザーが選択した応答の勝者を予測するモデルを構築しています。具体的には、ユーザーが選んだモデルの応答に基づいて、クラス分類問題を解決します。データの不均衡が存在するため、SMOTE（Synthetic Minority Over-sampling Technique）を用いて不均衡データのリサンプリングも行っています。

### 使用する手法とライブラリ
Notebookでは以下の手法とライブラリが使用されています：
- **NumPyとPandas**: 配列操作やデータフレームの操作に使用され、データの前処理と可視化に役立ちます。
- **Scikit-learn**: モデルの構築と評価に用いられ、特に以下が使用されています。
  - `TfidfVectorizer`: テキストデータをベクトル化するために使用。
  - `LogisticRegression`, `RandomForestClassifier`, `GradientBoostingClassifier`: 様々な分類モデルの構築に使用。
  - `GridSearchCV`: ハイパーパラメータの最適化に利用。
  - `SMOTE`: 不均衡データのオーバーサンプリングに使用。
  - `classification_report`, `log_loss`, `confusion_matrix`, `roc_curve`: モデルの評価指標。
  
- **MatplotlibとSeaborn**: データの可視化によく使われ、クラス分布のプロットや混同行列、ROC曲線などが表示されます。

### プロセスの概要
1. **データ収集**: トレーニングデータとテストデータをCSVファイルから読み込み。
2. **データ理解**: データの情報を確認し、特にクラスの分布などを可視化。
3. **データ準備**: テキストデータのクリーニング（URLやHTMLタグの除去、ストップワードの削除）を行い、テキストデータをTF-IDFベクトルに変換。
4. **特長エンジニアリング**: 応答の長さや自己強調に関する特徴量を追加。
5. **モデルの選択と評価**: モデルをトレーニング、バリデーションセットで評価、最適なハイパーパラメータを探索。
6. **予測**: 最良モデルを用いてテストデータに対する予測確率を生成し、最終的な提出ファイルを作成。

最終的に、テストデータの予測結果を含むCSVファイルを作成し、提出準備を完了します。

---


# 用語概説 
以下に、機械学習・深層学習の初心者がつまずきそうな専門用語の簡単な解説をリストします。特に、初心者にとってマイナーなものや実務経験がないと馴染みが薄い用語、ノートブック特有のドメイン知識に焦点を当てました。

1. **TF-IDF（Term Frequency-Inverse Document Frequency）**:
   - 文書における単語の重要性を評価する手法。単語の出現頻度とその単語が他の文書に出現する頻度を考慮して、情報をベクトルで表現する。

2. **SMOTE（Synthetic Minority Over-sampling Technique）**:
   - 不均衡データ処理手法の一つ。少数派クラスのサンプルを合成することでサンプル数を増やし、クラスバランスを改善する。近傍のデータポイントを元に新しいサンプルを生成する。

3. **クラス分布**:
   - データの各クラス（ラベル）がどのように分布しているかを示す。例えば、2つのクラスがある場合、クラスAとクラスBのデータポイントの割合を示す。

4. **バイアス（Bias）**:
   - モデルやデータ処理において、特定の特徴や傾向に偏りが生じる現象。例えば、どちらかのモデルの応答が常に選ばれることがある場合、順序バイアスが存在する。

5. **ポジションバイアス**:
   - 対応する応答がリスト上でどの位置にあるかによる影響。例えば、最初に提示された応答が好まれる傾向がある。

6. **セルフエンハンスメント**:
   - 自己宣伝とも言われ、モデルが自身を過剰に良く見せるために用いる表現やキーワード。例えば、「私は最高の〇〇です」といった表現を含む。

7. **ダミー変数（Dummy Variables）**:
   - カテゴリデータを数値データに変換するために使用される変数。例えば、「モデルA」と「モデルB」というカテゴリをそれぞれ0と1の値に変換して扱う。

8. **グリッドサーチ（Grid Search）**:
   - ハイパーパラメータの最適化手法。指定された複数のハイパーパラメータの組み合わせを試し、その中から最適なモデルを選ぶ。

9. **混同行列**:
   - モデルの予測性能を示す行列。真陽性、偽陽性、真陰性、偽陰性の数を用いて、モデルの分類結果を視覚化する。

10. **ROC曲線（Receiver Operating Characteristic Curve）**:
   - 分類モデルの性能を評価するためのグラフ。偽陽性率と真陽性率をプロットしたもので、曲線の下の面積（AUC）が大きいほどモデルの性能が良いとされる。

これらの用語は、初心者が理解するのが難しいが、特定のコンテキストや実務経験があれば意味を把握できるものです。

---


In [ ]:
# このPython 3環境には、多くの便利な分析ライブラリがインストールされています
# これはkaggle/python Dockerイメージによって定義されています: https://github.com/kaggle/docker-python
# たとえば、以下はいくつかの便利なパッケージのロード方法です

import numpy as np # 線形代数のためのライブラリ
import pandas as pd # データ処理とCSVファイルの入出力 (例: pd.read_csv)

# 入力データファイルは、読み取り専用の"../input/"ディレクトリにあります
# たとえば、これを実行すると（クリックするかShift+Enterを押すことで）入力ディレクトリ下の全ファイルが一覧表示されます

import os
for dirname, _, filenames in os.walk('/kaggle/input'): # 指定したディレクトリ内を再帰的に探索します
    for filename in filenames: # 見つかったファイルのリストをループします
        print(os.path.join(dirname, filename)) # 各ファイルのフルパスを出力します

# 現在のディレクトリ (/kaggle/working/) には最大で20GBのデータを書き込むことができ、
# このデータは「保存してすべて実行」を使用してバージョンを作成するときに出力として保存されます
# また、一時ファイルを/kaggle/temp/に書き込むこともできますが、これは現在のセッションの外で保存されません

In [ ]:
# 必要なライブラリをインポートします
import numpy as np  # 数値計算用のライブラリ
import pandas as pd  # データ処理用のライブラリ
import re  # 正規表現操作用のライブラリ
from sklearn.feature_extraction.text import TfidfVectorizer  # TF-IDFベクトル化用
from sklearn.model_selection import train_test_split, GridSearchCV  # データ分割およびグリッドサーチ用
from sklearn.metrics import classification_report, log_loss, confusion_matrix, roc_curve, auc  # モデル評価用のメトリクス
from imblearn.over_sampling import SMOTE  # 不均衡データを扱うためのオーバーサンプリング手法
from sklearn.linear_model import LogisticRegression  # ロジスティック回帰モデル
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier  # ランダムフォレストおよび勾配ブースティングモデル
import matplotlib.pyplot as plt  # グラフ描画用のライブラリ
import seaborn as sns  # データ可視化用のライブラリ

print('全てのライブラリがインポートされました')  # インポートが完了したことを表示します

In [ ]:
# データ収集
train_file = '/kaggle/input/lmsys-chatbot-arena/train.csv'  # トレーニングデータのファイルパス
test_file = '/kaggle/input/lmsys-chatbot-arena/test.csv'  # テストデータのファイルパス
train_data = pd.read_csv(train_file)  # トレーニングデータをCSVファイルから読み込みます
test_data = pd.read_csv(test_file)  # テストデータをCSVファイルから読み込みます

In [ ]:
# データ理解
print("トレーニングデータセット情報:")
print(train_data.info())  # トレーニングデータセットの情報を表示します
print("\n最初の数行:")
print(train_data.head())  # トレーニングデータの最初の数行を表示します

print("\nテストデータセット情報:")
print(test_data.info())  # テストデータセットの情報を表示します
print("\n最初の数行:")
print(test_data.head())  # テストデータの最初の数行を表示します

# リサンプリング前のクラス分布を可視化
plt.figure(figsize=(12, 6))  # グラフのサイズを設定
sns.countplot(x=train_data['winner_model_a'])  # 'winner_model_a'のクラス分布をカウントプロットします
plt.title("リサンプリング前のクラス分布")  # グラフのタイトルを設定
plt.show()  # グラフを表示します

In [ ]:
# データ準備
def clean_text(text, stop_words):  # テキストをクリーンアップする関数
    text = re.sub(r'\[.*?\]', '', text)  # 角括弧内のテキストを削除
    text = re.sub(r'http\S+|www.\S+', '', text)  # URLを削除
    text = re.sub(r'<.*?>+', '', text)  # HTMLタグを削除
    text = re.sub(r'[^a-zA-Z\s]', '', text)  # アルファベットとスペース以外の文字を削除
    text = text.lower()  # すべての文字を小文字に変換
    text = ' '.join(word for word in text.split() if word not in stop_words)  # ストップワードを削除
    return text  # クリーンアップされたテキストを返す

stop_words = set()  # ストップワードのセットを初期化

# テキストデータをクリーンアップ
train_data['prompt'] = train_data['prompt'].apply(lambda x: clean_text(x, stop_words))  # 'prompt'列のテキストをクリーンアップ
train_data['response_a'] = train_data['response_a'].apply(lambda x: clean_text(x, stop_words))  # 'response_a'列のテキストをクリーンアップ
train_data['response_b'] = train_data['response_b'].apply(lambda x: clean_text(x, stop_words))  # 'response_b'列のテキストをクリーンアップ

test_data['prompt'] = test_data['prompt'].apply(lambda x: clean_text(x, stop_words))  # テストデータの'prompt'列をクリーンアップ
test_data['response_a'] = test_data['response_a'].apply(lambda x: clean_text(x, stop_words))  # テストデータの'response_a'列をクリーンアップ
test_data['response_b'] = test_data['response_b'].apply(lambda x: clean_text(x, stop_words))  # テストデータの'response_b'列をクリーンアップ

In [ ]:
# テキストデータをベクトル化
vectorizer = TfidfVectorizer(max_features=1000)  # 最大1000特徴を持つTF-IDFベクトルライザーを作成
train_text = train_data['prompt'] + ' ' + train_data['response_a'] + ' ' + train_data['response_b']  # トレーニングテキストを結合
test_text = test_data['prompt'] + ' ' + test_data['response_a'] + ' ' + test_data['response_b']  # テストテキストを結合

X_train_text = vectorizer.fit_transform(train_text)  # トレーニングデータをTF-IDFベクトル化
X_test_text = vectorizer.transform(test_text)  # テストデータをTF-IDFベクトル化

# 表現のバイアス - 応答の長さと長さの差を追加
train_data['response_a_length'] = train_data['response_a'].apply(len)  # 'response_a'の長さを計算
train_data['response_b_length'] = train_data['response_b'].apply(len)  # 'response_b'の長さを計算
test_data['response_a_length'] = test_data['response_a'].apply(len)  # テストデータの'response_a'の長さを計算
test_data['response_b_length'] = test_data['response_b'].apply(len)  # テストデータの'response_b'の長さを計算
train_data['length_diff'] = train_data['response_a_length'] - train_data['response_b_length']  # 応答長の差を計算
test_data['length_diff'] = test_data['response_a_length'] - test_data['response_b_length']  # テストデータの応答長の差を計算

# ポジションバイアス - ポジションバイアスの特徴を追加
train_data['position_bias_a'] = 0  # 'response_a'が常に最初であると仮定
train_data['position_bias_b'] = 1  # 'response_b'が常に2番目であると仮定
test_data['position_bias_a'] = 0
test_data['position_bias_b'] = 1

# セルフエンハンスメントバイアス - セルフエンハンスメント検出の特徴を追加
def detect_self_enhancement(text):  # セルフエンハンスメントを検出する関数
    keywords = ['best', 'better', 'excellent', 'superior', 'number one']  # キーワードリスト
    for keyword in keywords:  # キーワードの各要素について
        if keyword in text:  # キーワードがテキストに含まれているかチェック
            return 1  # 見つかった場合は1を返す
    return 0  # 見つからなかった場合は0を返す

train_data['self_enhancement_a'] = train_data['response_a'].apply(detect_self_enhancement)  # 'response_a'のセルフエンハンスメントを検出
train_data['self_enhancement_b'] = train_data['response_b'].apply(detect_self_enhancement)  # 'response_b'のセルフエンハンスメントを検出
test_data['self_enhancement_a'] = test_data['response_a'].apply(detect_self_enhancement)  # テストデータの'response_a'のセルフエンハンスメントを検出
test_data['self_enhancement_b'] = test_data['response_b'].apply(detect_self_enhancement)  # テストデータの'response_b'のセルフエンハンスメントを検出

In [ ]:
# カテゴリカル特徴のエンコーディング
categorical_columns = ['model_a', 'model_b']  # カテゴリカル列のリスト
for column in categorical_columns:  # 各カテゴリカル列について
    if column not in test_data.columns:  # テストデータに列が存在しない場合
        test_data[column] = 'missing'  # 存在しない列には'missing'という値を設定
train_data_encoded = pd.get_dummies(train_data, columns=categorical_columns)  # トレーニングデータをダミー変数にエンコード
test_data_encoded = pd.get_dummies(test_data, columns=categorical_columns)  # テストデータをダミー変数にエンコード
train_data_encoded, test_data_encoded = train_data_encoded.align(test_data_encoded, join='left', axis=1, fill_value=0)  # データセットの整合性を確保

# 欠損列の処理
test_data_encoded.drop(columns=['winner_model_a', 'winner_model_b', 'winner_tie'], errors='ignore', inplace=True)  # テストデータから無視可能な列を削除

# 非数値列の削除
non_numeric_columns = train_data_encoded.select_dtypes(exclude=[np.number]).columns  # 非数値列を選択
train_data_encoded.drop(columns=non_numeric_columns, inplace=True)  # トレーニングデータから非数値列を削除
test_data_encoded.drop(columns=non_numeric_columns, inplace=True)  # テストデータから非数値列を削除

# すべての特徴をトレーニングおよびテストセットに結合
X_train_combined = np.hstack((X_train_text.toarray(), train_data_encoded.drop(columns=['winner_model_a', 'winner_model_b', 'winner_tie']).values))  # トレーニングセットを結合
X_test_combined = np.hstack((X_test_text.toarray(), test_data_encoded.values))  # テストセットを結合
X = X_train_combined  # 特徴行列Xにトレーニングセットを設定
y = train_data_encoded['winner_model_a']  # 目的変数yにトレーニングデータの'winner_model_a'を設定

In [ ]:
# モデリング
# データのリサンプリング
smote = SMOTE(random_state=42)  # SMOTEインスタンスを作成（ランダムシードを42に設定）
X_resampled, y_resampled = smote.fit_resample(X, y)  # データをリサンプリング

# リサンプリング後のクラス分布を可視化
plt.figure(figsize=(12, 6))  # グラフのサイズを設定
sns.countplot(x=y_resampled)  # リサンプリング後のクラス分布をカウントプロット
plt.title("リサンプリング後のクラス分布")  # グラフのタイトルを設定
plt.show()  # グラフを表示します

In [ ]:
# リサンプルされたデータをトレーニングセットとバリデーションセットに分割
X_train, X_val, y_train, y_val = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)  # データを80%トレーニング、20%バリデーションに分割

In [ ]:
# ハイパーパラメータの調整とモデル選択
models = {  # 使用するモデルの設定
    'Logistic Regression': {
        'model': LogisticRegression(random_state=42, max_iter=1000),  # ロジスティック回帰モデル
        'params': {'C': [0.01, 0.1, 1, 10, 100]}  # ハイパーパラメータの候補
    },
    'Random Forest': {
        'model': RandomForestClassifier(random_state=42),  # ランダムフォレストモデル
        'params': {'n_estimators': [50, 100, 200], 'max_depth': [10, 20, 30]}  # ハイパーパラメータの候補
    },
    'Gradient Boosting': {
        'model': GradientBoostingClassifier(random_state=42),  # 勾配ブースティングモデル
        'params': {'learning_rate': [0.01, 0.1, 0.2], 'n_estimators': [100, 200]}  # ハイパーパラメータの候補
    }
}

best_models = {}  # 最良モデルを格納する辞書

for model_name, config in models.items():  # 各モデルについて
    grid_search = GridSearchCV(estimator=config['model'], param_grid=config['params'], cv=5, scoring='neg_log_loss', verbose=2, n_jobs=-1)  # グリッドサーチを設定
    grid_search.fit(X_train, y_train)  # トレーニングデータでモデルを学習
    best_models[model_name] = grid_search.best_estimator_  # 最良モデルを保存
    print(f"最良の{model_name}モデル: {grid_search.best_params_}")  # 最良ハイパーパラメータを出力

In [ ]:
# 評価
for model_name, model in best_models.items():  # 最良モデルの各名称とモデルについて
    y_val_pred = model.predict(X_val)  # バリデーションセットに対する予測を行う
    y_val_pred_proba = model.predict_proba(X_val)  # バリデーションセットに対する予測確率を計算
    print(f"バリデーションセットの分類レポート ({model_name}):")
    print(classification_report(y_val, y_val_pred, zero_division=1))  # 分類レポートを表示
    print(f"ログ損失 ({model_name}): {log_loss(y_val, y_val_pred_proba)}")  # ログ損失を表示

    # 混同行列
    cm = confusion_matrix(y_val, y_val_pred)  # 混同行列を計算
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')  # 混同行列をヒートマップで表示
    plt.title(f'混同行列 ({model_name})')  # タイトルを設定
    plt.xlabel('予測値')  # x軸のラベルを設定
    plt.ylabel('真の値')  # y軸のラベルを設定
    plt.show()  # グラフを表示

    # ROC曲線
    fpr, tpr, _ = roc_curve(y_val, y_val_pred_proba[:, 1])  # 偽陽性率と真陽性率を計算
    roc_auc = auc(fpr, tpr)  # ROC曲線の下の面積を計算
    plt.figure()
    plt.plot(fpr, tpr, color='darkorange', lw=2, label='ROC曲線 (面積 = %0.2f)' % roc_auc)  # ROC曲線をプロット
    plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')  # 対角線をプロット
    plt.xlim([0.0, 1.0])  # x軸の範囲を設定
    plt.ylim([0.0, 1.05])  # y軸の範囲を設定
    plt.xlabel('偽陽性率')  # x軸のラベルを設定
    plt.ylabel('真陽性率')  # y軸のラベルを設定
    plt.title(f'受信者動作特性 ({model_name})')  # タイトルを設定
    plt.legend(loc="lower right")  # 凡例を表示
    plt.show()  # グラフを表示

In [ ]:
# 最良モデルを用いたデプロイメント (例: ロジスティック回帰)
best_lr_model = best_models['Logistic Regression']  # 最良のロジスティック回帰モデルを取得
test_predictions_proba = best_lr_model.predict_proba(X_test_combined)  # テストデータに対する予測確率を計算
submission_df = pd.DataFrame({
    'id': test_data['id'],  # テストデータのIDを追加
    'winner_model_a': test_predictions_proba[:, 0],  # モデルAの予測確率を追加
    'winner_model_b': test_predictions_proba[:, 1],  # モデルBの予測確率を追加
    'winner_tie': 0.0  # バイナリ分類であると仮定し、引き分けの列を0.0に設定
})
submission_df.to_csv('submission.csv', index=False)  # 提出用ファイルをCSVとして保存
print(submission_df.head())  # 提出用ファイルの先頭行を表示
print("提出ファイルが正常に保存されました。")  # 保存完了のメッセージを表示